# Stellar Mass Density Radial Profiles at 9 arcsec beam + 2 arcsec pixel resolution

Isaac Cheng - December 2021


In [1]:
SIG_STAR_PATH = "/arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities/"
%cd "$SIG_STAR_PATH"

from multiprocessing import Pool
from itertools import repeat
import re

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dill
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
# 
# Load my own packages
# 
import sys
sys.path.append("/arc/home/IsaacCheng/coop_f2021/packages")
import fits_utils as fu
import plot_utils as pu
import radial_profile_utils as rpu
from radial_profile import RadialProfile
# 
GALDIST = 16.5 * u.Mpc  # distance to Virgo cluster centre

/arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities


In [2]:
I_THRESHOLD = 80  # degree. If inclination >= I_THRESHOLD, means high-i galaxy

# ----------------------------------------------------------------------------------------
GALAXIES = np.array(
    [
        "IC3392",
        # "NGC4189",  # ignored flag map, but bad results
        "NGC4192",
        "NGC4216",  # has unflagged foreground star
        "NGC4222",
        "NGC4254",
        "NGC4294",
        "NGC4298",
        "NGC4299",
        "NGC4302",
        "NGC4321",
        "NGC4330",
        "NGC4351",
        "NGC4380",
        "NGC4383",
        "NGC4388",
        "NGC4396",
        "NGC4402",
        "NGC4405",
        "NGC4419",
        "NGC4424",
        "NGC4450",
        "NGC4501",
        "NGC4522",
        "NGC4532",
        "NGC4535",
        "NGC4548",
        "NGC4567",
        "NGC4568",
        "NGC4569",
        "NGC4579",
        "NGC4580",
        # "NGC4606",  # flag map too intrusive
        "NGC4607",
        "NGC4651",
        "NGC4654",
        "NGC4689",
        "NGC4694",
    ]
)
INCLINATIONS = np.array(
    [
        68,
        # 42,  # ignored flag map, but bad results
        83,
        90,  # has unflagged foreground star
        90,
        39,
        74,
        52,
        14,
        90,
        32,
        90,
        48,
        61,
        56,
        83,
        83,
        80,
        46,
        74,
        61,
        51,
        65,
        82,
        64,
        48,
        37,
        49,
        70,
        69,
        40,
        46,
        # 69,  # flag map too intrusive
        90,
        53,
        61,
        38,
        62,
    ]
)  # degrees
POSITION_ANGLES = np.array(
    [
        219,
        # 70,  # ignored flag map, but bad results
        333,
        20,  # has unflagged foreground star
        238,
        243,
        151,
        132,
        128,
        356,
        280,
        238,
        251,
        158,
        17,
        271,
        304,
        270,
        18,
        131,
        274,
        170,
        320,
        35,
        159,
        12,
        318,
        251,
        211,
        203,
        273,
        337,
        # 38,  # flag map too intrusive
        2,
        75,
        300,
        341,
        323,
    ]
)  # degrees
# ----------------------------------------------------------------------------------------
# N.B. "high inclination" means inclination >= 80 degrees
HIGH_I_GALAXIES = np.array(
    [
        "NGC4192",
        "NGC4216",
        "NGC4222",
        "NGC4302",
        "NGC4330",
        "NGC4388",
        "NGC4396",
        "NGC4402",
        "NGC4522",
        "NGC4607",
    ]
)
HIGH_I_INCLINATIONS = np.array([83, 90, 90, 90, 90, 83, 83, 80, 82, 90,])  # degrees
HIGH_I_POSITION_ANGLES = np.array(
    [333, 20, 238, 356, 238, 271, 304, 270, 35, 2]
)  # degrees
# ----------------------------------------------------------------------------------------
HIGH_I_HEIGHTS_ARCSEC = np.array(
    [80, 60, 25, 55, 38, 65, 43, 45, 40, 70,]
)  # arcsec, for radial profiles
# ----------------------------------------------------------------------------------------
VCC_GALAXIES = np.array(
    [
        1126,  # IC3392
        # 89,  # NGC4189, ignored flag map, but bad results
        92,  # NGC4192
        167,  # NGC4216, has unflagged foreground star
        187,  # NGC4222
        307,  # NGC4254
        465,  # NGC4294
        483,  # NGC4298
        491,  # NGC4299
        497,  # NGC4302
        596,  # NGC4321
        630,  # NGC4330
        692,  # NGC4351
        792,  # NGC4380
        801,  # NGC4383
        836,  # NGC4388
        865,  # NGC4396
        873,  # NGC4402
        874,  # NGC4405
        958,  # NGC4419
        979,  # NGC4424
        1110,  # NGC4450
        1401,  # NGC4501
        1516,  # NGC4522
        1554,  # NGC4532
        1555,  # NGC4535
        1615,  # NGC4548
        1673,  # NGC4567
        1676,  # NGC4568
        1690,  # NGC4569
        1727,  # NGC4579
        1730,  # NGC4580
        # 1859,  # NGC4606, flag map too intrusive
        1868,  # NGC4607
        -100,  # NGC4651 (EVCC number is 1102, cannot use EVCC number)
        1987,  # NGC4654
        2058,  # NGC4689
        2066,  # NGC4694
    ]
)
# ----------------------------------------------------------------------------------------
# ! WARNING: DO NOT UNCOMMENT NGC4189 OR NGC4606 IF USING THIS AS THEY WERE NOT CONSIDERED !
DECR_MEAN_M_DENSITY_IDX = np.array(
    [
        29, 18, 21, 30, 27, 26, 28, 0, 17, 35, 25, 9, 12, 34, 32, 16, 14, 24, 4, 20, 11, 33, 19, 6, 7, 23, 2, 5, 22, 3, 1, 8, 13, 10, 31, 15,
    ]
)  # only considered finite SNRs >= 30 and treated each Voronoi bin as 1 pixel. All data corrected for inclination
# ! WARNING: DO NOT UNCOMMENT NGC4189 OR NGC4606 IF USING THIS AS THEY WERE NOT CONSIDERED !
DECR_MEDIAN_M_DENSITY_IDX = np.array(
    [
        30, 29, 21, 26, 0, 17, 18, 27, 35, 9, 34, 12, 25, 28, 32, 11, 24, 19, 20, 14, 4, 16, 33, 7, 6, 5, 13, 22, 10, 3, 23, 2, 8, 15, 1, 31
    ]
)  # only considered finite SNRs >= 30 and treated each Voronoi bin as 1 pixel. All data corrected for inclination
# ----------------------------------------------------------------------------------------

In [3]:
from radio_beam import Beam

def get_9as_rp(
    galaxy, i, pa,
    include_bad=None,  # overrides default include_bad
    snr_cutoff=None,
    max_snr_annuli=40,
    n_annuli=None,
    avg_function="mean",
    min_width=None,
    bootstrap_errs=True,
    bootstrap_seed=1234,
):
    # 
    # Load stellar mass density data
    # 
    galpath = f"/arc/home/IsaacCheng/coop_f2021/galaxies/{galaxy}/"
    galaxy_infile = galpath + f"{galaxy}_regBin_extinctionCorr_9as_ugiz_Sigma-star_noNorm_i_corr.pkl"
    with open(galaxy_infile, "rb") as f:
        file = dill.load(f)
        # gal_wcs = file["wcs_binned"]
        # gal_wcs.array_shape = file["wcs_binned_array_shape"]
        M_density = file["stellar_mass_density"]
        M_density_err = file["stellar_mass_density_err"]
        file = None  # free memory
    #
    # Load molecular gas density data for header
    #
    if galaxy == "NGC4321":  # no 9 arcsec data since native is 10 arcsec
        Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3.1/native/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_round_mom0_Kkms-1.fits"
        Ico_unc_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3.1/native/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_round_mom0_unc.fits"
    else:
        Ico_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_9as_round_mom0_Kkms-1.fits"
        Ico_unc_path = f"/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/{galaxy}/{galaxy}_7m+tp_co21_pbcorr_9as_round_mom0_unc.fits"
    Ico_data, gal_header = fits.getdata(Ico_path, header=True)
    Ico_unc = fits.getdata(Ico_unc_path)  # uncertainty
    gal_wcs = WCS(gal_header)
    #
    # Get annulus width in degree (just to store)
    #
    beam = Beam.from_fits_header(gal_header)
    annulus_width_arcsec = np.max([beam.major.to(u.arcsec).value, beam.minor.to(u.arcsec).value])
    #
    # Get radial profile parameters
    #
    centre = gal_wcs.pixel_to_world(Ico_data.shape[1] / 2, Ico_data.shape[0] / 2)
    if i >= I_THRESHOLD:
        if include_bad is None:
            include_bad = False
        high_i_height = HIGH_I_HEIGHTS_ARCSEC[galaxy == HIGH_I_GALAXIES] * u.arcsec
    else:
        if include_bad is None:
            include_bad = True  # only include NaNs if low-i galaxy
        high_i_height = None
    #
    # Make radial profile
    #
    galaxy_rp = RadialProfile(M_density, centre, i, pa, noise=M_density_err)
    galaxy_rp = galaxy_rp.calc_radial_profile(
        i_threshold=I_THRESHOLD,
        n_annuli=n_annuli,
        snr_cutoff=snr_cutoff,
        max_snr_annuli=max_snr_annuli,
        min_width=min_width,
        wcs=gal_wcs,
        header=gal_header,
        include_bad=include_bad,
        method="exact",
        func=avg_function,
        is_radio=True,
        high_i_height=high_i_height,
        bootstrap_errs=bootstrap_errs,
        bootstrap_seed=bootstrap_seed,
    )
    return galaxy_rp, centre, annulus_width_arcsec

In [4]:
# 
# Load VERTICO gas density radial profiles (to get n_annuli)
# 
with open("/arc/home/IsaacCheng/coop_f2021/galaxies/gas_densities/gas_density_i_corr_allVERTICO.pkl", "rb") as f:
    file = dill.load(f)
    vertico_gals = file["galaxies"]
    vertico_inclinations = file["inclinations"]
    vertico_pas = file["position_angles"]
    vertico_rps = file["radial_profiles"]
    file = None  # free memory
n_annulis = []
for g, i, p, rp in zip(vertico_gals, vertico_inclinations, vertico_pas, vertico_rps):
    if g not in GALAXIES:
        continue
    n_annulis.append(len(rp.annuli))
print(len(n_annulis))
print(n_annulis)  # here, n_annulis correspond to the alphabetized GALAXIES (make sure all lists/arrays correspond element-wise!)
print(GALAXIES)

36
[3, 19, 21, 9, 16, 3, 7, 6, 18, 15, 9, 5, 5, 5, 10, 8, 13, 3, 4, 4, 7, 7, 7, 4, 12, 11, 6, 5, 8, 11, 4, 7, 6, 8, 8, 5]
['IC3392' 'NGC4192' 'NGC4216' 'NGC4222' 'NGC4254' 'NGC4294' 'NGC4298'
 'NGC4299' 'NGC4302' 'NGC4321' 'NGC4330' 'NGC4351' 'NGC4380' 'NGC4383'
 'NGC4388' 'NGC4396' 'NGC4402' 'NGC4405' 'NGC4419' 'NGC4424' 'NGC4450'
 'NGC4501' 'NGC4522' 'NGC4532' 'NGC4535' 'NGC4548' 'NGC4567' 'NGC4568'
 'NGC4569' 'NGC4579' 'NGC4580' 'NGC4607' 'NGC4651' 'NGC4654' 'NGC4689'
 'NGC4694']


In [4]:
with Pool(processes=None) as p:
    radial_profiles, centers, annulus_widths_arcsec= zip(
        *p.starmap(
            get_9as_rp, zip(GALAXIES, INCLINATIONS, POSITION_ANGLES, repeat(False), repeat(0))
        )
    )
radial_profiles = list(radial_profiles)
centers = list(centers)
annulus_widths_arcsec = list(annulus_widths_arcsec)
print("Done")

: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]WARNING: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformation has more axes (2) than the image it is associated with (0) [astropy.wcs.wcs]: FITSFixedWarning: The WCS transformatio

Info: using provided WCS instead of converting header to WCSInfo: using provided WCS instead of converting header to WCS

Info: using provided WCS instead of converting header to WCS

: FITSFixedWarning: 'obsfix' made the change 'Observatory coordinates incomplete'. [astropy.wcs.wcs]
: FITSFixedWarning: 'datfix' made the change 'Set MJD-OBS to 58184.283297 from DATE-OBS'. [astropy.wcs.wcs]

Info: using provided WCS instead of converting header to WCS

...

Stopped creating ellipses/annuli after 20 iterations
Average SNR of most recent successful fit: 26.33390143937507
Average SNR of next annulus: nan
Iter 18: average SNR = 91.90572638574632
Iter 16: average SNR = 75.48220009102357
Iter 19: average SNR = 91.46196430062422
Iter 17: average SNR = 73.62353087424249
Iter 20: average SNR = 91.43888236348117
Iter 18: average SNR = 98.4503844354079
Iter 21: average SNR = 95.69594463733624
Iter 19: average SNR = 72.67902076734862
Iter 22: average SNR = 58.01828174991169
Iter 20: average SNR = 34.729003222003556
Iter 23: average SNR = 15.817676976902453
Iter 21: average SNR = 16.83739980036709


/arc/home/IsaacCheng/coop_f2021/packages/radial_profile_utils.py:607: RuntimeWarning: invalid value encountered in double_scalars
  avg_signal = avg_signal / np.nansum(signal_area_mask)
/arc/home/IsaacCheng/coop_f2021/packages/radial_profile_utils.py:613: RuntimeWarning: invalid value encountered in double_scalars
  avg_noise = np.sqrt(avg_noise) / np.nansum(noise_area_mask)
/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Stopped creating rectangles after 23 iterations
Average SNR of most recent successful fit: 15.817676976902453
Average SNR of next rectangle: nan


/arc/home/IsaacCheng/coop_f2021/packages/radial_profile_utils.py:607: RuntimeWarning: invalid value encountered in double_scalars
  avg_signal = avg_signal / np.nansum(signal_area_mask)
/arc/home/IsaacCheng/coop_f2021/packages/radial_profile_utils.py:613: RuntimeWarning: invalid value encountered in double_scalars
  avg_noise = np.sqrt(avg_noise) / np.nansum(noise_area_mask)
/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Stopped creating rectangles after 21 iterations
Average SNR of most recent successful fit: 16.83739980036709
Average SNR of next rectangle: nan
Done


In [5]:
sig_star_outfile = SIG_STAR_PATH + "stellar_mass_density_9as_i_corr_NGVS-VERTICO_noNorm_noIncludeBad_fitUntilAllNaNs.pkl"
with open(sig_star_outfile, "wb") as f:
    dill.dump(
        {
            "galaxies": GALAXIES,
            "inclinations": INCLINATIONS,
            "position_angles": POSITION_ANGLES,
            "radial_profiles": radial_profiles,
            "centers": centers,
            "annulus_widths_arcsec": annulus_widths_arcsec,
            # "n_annulis": n_annulis,
        },
        f
    )
print("Pickled", sig_star_outfile)

Pickled /arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities/stellar_mass_density_9as_i_corr_NGVS-VERTICO_noNorm_noIncludeBad_fitUntilAllNaNs.pkl


# Plot individual radial profiles


In [4]:
PLOT_INCLUDE_BAD = False  # determines pickle file to use
PLOT_FIT_UNTIL_ALL_NANS = True  # if True, use file that fits annuli until an annulus is all NaN
# 
if PLOT_INCLUDE_BAD:
    incl_bad_str = ""
else:
    incl_bad_str = "_noIncludeBad"
if PLOT_FIT_UNTIL_ALL_NANS:
    fit_until_all_nans_str = "_fitUntilAllNaNs"
else:
    fit_until_all_nans_str = ""
# 
# Load radial profiles
# 
sig_star_outfile = SIG_STAR_PATH + f"stellar_mass_density_9as_i_corr_NGVS-VERTICO_noNorm{incl_bad_str}{fit_until_all_nans_str}.pkl"
print("Loading", sig_star_outfile)
with open(sig_star_outfile, "rb") as f:
    file = dill.load(f)
    galaxies = file["galaxies"]
    inclinations = file["inclinations"]
    position_angles = file["position_angles"]
    centers = file["centers"]
    radial_profiles = file["radial_profiles"]
    annulus_widths_arcsec = file["annulus_widths_arcsec"]
    file = None  # free memory
avg_function = radial_profiles[0].rp_options["func"]
# 
# Calculate radii of annuli in kpc
# 
radius_kpc = []
for galname in galaxies:
    galidx = np.where(galaxies == galname)[0]
    if galidx.size != 1:
        raise ValueError(f"More than 1 galaxy matched with name {galname} at indices {galidx}")
    galidx = galidx[0]
    gal_wcs = WCS(radial_profiles[galidx].rp_options["header"])
    px_to_kpc = fu.calc_pc_per_px(gal_wcs, GALDIST)[0][0] / 1000
    radius_kpc.append(radial_profiles[galidx].radii * px_to_kpc)
#
I_THRESHOLD = 80  # degrees

Loading /arc/home/IsaacCheng/coop_f2021/galaxies/stellar_mass_densities/stellar_mass_density_9as_i_corr_NGVS-VERTICO_noNorm_noIncludeBad_fitUntilAllNaNs.pkl


In [5]:
for galname, i, gal_rp, r_kpc, gal_cen in zip(galaxies, inclinations, radial_profiles, radius_kpc, centers):
    # if galname != "NGC4222":
    #     continue
    # print(gal_cen.to_string('hmsdms'))
    is_high_i = i >= I_THRESHOLD
    gal_wcs = WCS(gal_rp.rp_options["header"])
    #
    fig = plt.figure(figsize=mpl.figure.figaspect(1.5))
    ax1 = fig.add_subplot(2, 1, 1, projection=gal_wcs)
    #
    # Plot data
    #
    # img1 = ax1.imshow(gal_rp.data, cmap=sns.cubehelix_palette(start=.5, rot=-.75, as_cmap=True))
    img1 = ax1.imshow(gal_rp.data, cmap="magma", norm=mpl.colors.LogNorm())
    cbar1 = fig.colorbar(img1, fraction=0.045)
    cbar1.set_label(r"$\Sigma_{\star}$ [$\rm M_\odot\;pc^{-2}$]")
    # Mark centre
    ax1.plot(*gal_wcs.world_to_pixel(gal_cen), marker="o", color="c", markersize=4)
    # Add annuli
    pu.add_annuli_RadialProfile(ax1, gal_rp)
    # Add scalebar
    scalebar_colour = "grey"
    scalebar_alpha = 1
    pu.add_scalebar(ax1, gal_wcs, dist=GALDIST, color=mpl.colors.to_rgba(scalebar_colour)[0:3] + (scalebar_alpha,))
    pu.add_scalebeam(ax1, gal_rp.rp_options["header"], ec="grey", alpha=1)
    ax1.set_xlim(0, gal_rp.data.shape[1])
    ax1.set_ylim(0, gal_rp.data.shape[0])
    ax1.set_xlabel("RA (J2000)")
    ax1.set_ylabel("Dec (J2000)")
    ax1.grid(False)
    ax1.set_aspect("equal")
    #
    ax2 = fig.add_subplot(2, 1, 2)
    ebar2 = ax2.errorbar(
        x=r_kpc,
        y=gal_rp.avg_data,
        yerr=gal_rp.avg_data_err,  # uncertainty in the mean/median from bootstrapping
        fmt="-o",
        markersize=4,
        color="k",
        ecolor="r",
        elinewidth=1,
        capsize=2,
    )
    ebar2[-1][0].set_linestyle("--")
    ax2.semilogy()
    # 
    # Add name of galaxy
    # 
    high_i_str = "*" if gal_rp.i >= I_THRESHOLD else ""
    ax2.text(
        0.9,
        0.9,
        re.sub(r"(\d+)", " \\1", galname) + high_i_str,
        c="k",
        ha="right",
        transform=ax2.transAxes,
    )
    ax2.set_xlabel("Radius [kpc]")
    ax2.set_ylabel(
        gal_rp.rp_options["func"].capitalize()
        + r" $\Sigma_{\star}$ [$\rm M_\odot\;pc^{-2}$]"
    )
    ax2.set_xlim(left=0)
    # pu.set_aspect(ax2, 0.9, logx=False, logy=True)
    #
    fig.tight_layout(pad=3.5)
    # fig.savefig(SIG_STAR_PATH + f"9as/{galname}_stellar_mass_density_9as_i_corr_noNorm{incl_bad_str}{fit_until_all_nans_str}.pdf")
    plt.close()
print("Done!")

/tmp/ipykernel_28920/638030849.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  cbar1 = fig.colorbar(img1, fraction=0.045)
/tmp/ipykernel_28920/638030849.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  cbar1 = fig.colorbar(img1, fraction=0.045)
/tmp/ipykernel_28920/638030849.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  cbar1 = fig.colorbar(img1, fraction=0.045)
/tmp/ipykernel_28920/638030849.py:15: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid

Done!
